In [2]:
from importlib.metadata import files
from pathlib import Path
import os
import numpy as np

In [3]:
def dump(lines, fname, path):
    if fname == 'skipdump':
        return
    with open(path+fname, 'a') as f:
        f.write(''.join(lines))

In [4]:
p1count = p2count = p3count = 0

def update_pcounts(fname, path):
    global p1count
    global p2count
    global p3count
    p1count = p2count = p3count = 0
    fsplit = fname.split('_')
    for file in os.listdir(path):
        if 'fscan' in file:
            p1count+=1
        if fsplit[1]+'_'+fsplit[2] in file:
            p2count+=1
        if fsplit[3]+'_'+fsplit[2] in file:
            p3count+=1

def getname(fname, phase):
    global p1count
    global p2count
    global p3count
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"fscan_{p1count}"
        case 2:
            if "none" in fname:
                return 'skipdump'
            return f"compress_{fsplit[1]}_{fsplit[2]}t_{p2count}"
        case 3:
            return f"transfer_{fsplit[3]}_{fsplit[2]}t_{p3count}"
    
in_dir = '../outputs/v3_results/out_exfil/'
out_dir = '../output/v4_results/out_exfil_parsed/'

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        update_pcounts(file, out_dir)
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if 'sys_exit_pidfd_getfd' in l:
                if phase > 0:
                    out_fname = getname(file, phase)
                    dump(phrases, out_fname, out_dir)
                    print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 4:
                    break
            elif 'sys_enter_pidfd_open' in l:
                readout = False
            elif readout:
                phrases.append(l)

FileNotFoundError: [Errno 2] No such file or directory: '../outputs/v3_results/out_exfil/'

In [13]:
p1count = p2count = p3count = 0

def update_pcounts(fname, path):
    global p1count
    global p2count
    global p3count
    p1count = p2count = p3count = 0
    fsplit = fname.split('_')
    for file in os.listdir(path):
        if 'asymm' in file:
            p1count+=1
        if fsplit[1]+'_'+fsplit[2] in file:
            p2count+=1
        if fsplit[3]+'_'+fsplit[2] in file:
            p3count+=1

def getname(fname, phase):
    global p1count
    global p2count
    global p3count
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"asymm_{p1count}"
        case 2:
            return f"symm_{fsplit[1]}_{fsplit[2]}b_{p2count}"
        case 3:
            return f"transfer_{fsplit[3]}_{fsplit[2]}t_{p3count}"
    
in_dir = '../outputs/v3_results/out_exec/'
out_dir = '../output/v4_results/out_exec_parsed/'

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        update_pcounts(file, out_dir)
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if 'sys_exit_pidfd_getfd' in l:
                if phase > 0:
                    out_fname = getname(file, phase)
                    dump(phrases, out_fname, out_dir)
                    print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 3:
                    break
            elif 'sys_enter_pidfd_open' in l:
                readout = False
            elif readout:
                phrases.append(l)

exfil_Salsa20_256_WA_none_2
Wrote 1648 to asymm_0
Wrote 59737 to symm_Salsa20_256b_0
exfil_AES_256_WA_none_2
Wrote 1725 to asymm_1
Wrote 59102 to symm_AES_256b_0
exfil_Salsa20_128_WA_none_3
Wrote 1648 to asymm_2
Wrote 59717 to symm_Salsa20_128b_0
exfil_AES_128_WA_none_3
Wrote 1720 to asymm_3
Wrote 59807 to symm_AES_128b_0
exfil_AES_256_O_none_2
Wrote 1722 to asymm_4
Wrote 57861 to symm_AES_256b_1
exfil_AES_256_O_default_3
Wrote 1724 to asymm_5
Wrote 58959 to symm_AES_256b_2
exfil_Salsa20_256_O_default_3
Wrote 1650 to asymm_6
Wrote 71662 to symm_Salsa20_256b_1
exfil_Salsa20_256_O_none_2
Wrote 1650 to asymm_7
Wrote 62941 to symm_Salsa20_256b_2
exfil_AES_128_WA_default_1
Wrote 1725 to asymm_8
Wrote 59704 to symm_AES_128b_1
exfil_Salsa20_128_WA_default_1
Wrote 1650 to asymm_9
Wrote 64904 to symm_Salsa20_128b_1
exfil_AES_256_WA_none_1
Wrote 1720 to asymm_10
Wrote 61205 to symm_AES_256b_3
exfil_Salsa20_256_WA_none_1
Wrote 1648 to asymm_11
Wrote 60509 to symm_Salsa20_256b_3
exfil_Salsa20_128_

In [163]:
def get_pname(idx):
    if idx == 1:
        return "asymm"
    if idx == 2:
        return "symm"

def get_count(phasename, path):
    count = 0
    for file in os.listdir(path):
        if phasename in file:
            count+=1
    return (count)

def get_fname(phase, path, original):
    oparts = original.split('_')
    if phase == 'asymm':
        count = get_count(phase, path)
        return f"{phase}_{count}"
    elif phase == 'symm':
        count = get_count(f"{phase}_{oparts[0]}_{oparts[1]}b", path)
        return f"{phase}_{oparts[0]}_{oparts[1]}b_{count}"

def dump_perf_data(data, ptimes, fname, run_iter, outdir):
    # print(f"{fname} {run_iter}")
    #     for key in data.keys():
    #         print(f"{key}: {len(data[key])}")
    processed_data = {}
    for key in data.keys():
        for pt in ptimes:
            if key in pt:
                phases = ptimes[pt]
        pstart = 0
        for i,time in enumerate(phases):
            if i == 0:
                pstart = time
                continue
            pname = get_pname(i)
            # pdata = [{t,data[key][t]} for t in data[key] if t<=time and t>=pstart]
            pdata = {}
            for t in data[key]:
                if t<=time and t>=pstart:
                    pdata.update({t:data[key][t]})
            if pname in processed_data:
                processed_data[pname].update({key:pdata})
            else:
                processed_data[pname] = {key:pdata}
            pstart = time
    print(fname)
    # print(processed_data)
    for ptype in processed_data:
        # ct = get_count(ptype, outdir)
        outfile = get_fname(ptype, outdir, fname)
        tmin = 100
        tmax = 0
        # print(processed_data[ptype])
        header="time,"
        keylist=[]
        for key in processed_data[ptype]:
            # print(key)
            # print(processed_data[ptype][key])
            tmin = list(processed_data[ptype][key].keys())[0] if list(processed_data[ptype][key].keys())[0] < tmin else tmin
            tmax = list(processed_data[ptype][key].keys())[-1] if list(processed_data[ptype][key].keys())[-1] > tmax else tmax
            keylist.append(key)
            header+=f"{key},"
            
        with open(f"{out_dir}/{outfile}","a") as f:
            f.write(header+'\n')
        # print(tmin)
        # print(tmax)
        # print(processed_data[ptype])
        for t in np.arange(tmin,tmax+0.09,0.1):
            t = round(t,1)
            vstring = f"{t},"
            for key in keylist:
                # print(processed_data[ptype][key])
                if t in processed_data[ptype][key].keys():
                    val = processed_data[ptype][key][t]
                else:
                    val = 0
                vstring+=f"{val},"
            with open(f"{out_dir}/{outfile}","a") as f:
                f.write(vstring+'\n')


In [169]:
## Perf counters processor
def save(time, value, metric, data):
    if metric in data.keys():
        data[metric].update({time: value})
    else:
        data.update({metric:{time: value}})
    return data

perf_dir = "../outputs/perf_results"
in_dir = f"{perf_dir}/out_exec/"
out_dir = f"{perf_dir}/out_exec_parsed/"

os.makedirs(out_dir, exist_ok=True)

# count = 0
# for file in os.listdir(perf_dir):
#     if 'phase' in file and 'exec_' in file:
#         with open(f"{perf_dir}/{file}",'r') as fp:
#             lines = fp.readlines()
#             # print(lines[-3:])
#             if 'str(elapsed_time)\n' in lines[-3:]:
#                 print(file)
#                 count+=1
# print(count)
runs = []
perf_mets = []
for file in os.listdir(in_dir):
    separator = "_"
    file_parts = file.split('_')
    file_run = separator.join(file_parts[1:5])
    if file_run not in runs:
        runs.append(file_run)
    if file[len(file_run)+6:-2] not in perf_mets:
        perf_mets.append(file[len(file_run)+6:-2])
print(runs)
print(perf_mets)
# runs = ['AES_128_O_none']
all_data = {}
for idx,run in enumerate(runs):
    # print(f"{idx} {run}")
    for i in range(2,5):
        data = {}
        phase_times = {}
        for met in perf_mets:
            file = f"exec_{run}_{met}_{i}"
            phase_file = file[:-2]+'_4_phase'
            with open(f"{perf_dir}/{phase_file}",'r') as fp:
                lines = fp.readlines()[-3:]
                if len(lines) > 3:
                    print(f"Something wrong here:{phase_file}")
                else:
                    p_time = []
                    for l in lines:
                        # p_time.append(math.ceil(float(l[:-2])*10)/10)
                        fl_num = round(float(l[:-2]),2)
                        if (fl_num*10)%1 > 0.4:
                            fl_num = int(fl_num*10+1)/10
                        p_time.append(fl_num)
                    phase_times[met] = p_time
                    # p1_time = round(float(lines[0][:-2]),1)
                    # p2_time = round(float(lines[1][:-2]),1)
                    # p3_time = round(float(lines[2][:-2]),1)
                    # print(f"{p1_time}, {p2_time}, {p3_time}")
                    # print(p_time)
            with open(f"{in_dir}/{file}",'r') as f:
                lines = f.readlines()
                for l in lines[2:]:
                    l=l.replace("<not counted>","0")
                    if "#" in l:
                        continue
                    # print(l)
                    # break
                    parts = l.split()
                    # print(round(float(parts[0]),1))
                    time = round(float(parts[0]),1)
                    # print(time)
                    value = int(parts[1].replace(',',''))
                    metric = parts[2]
                    data = save(time, value, metric, data)
                    # print(data)
        # dump_perf_data(data, phase_times, run, i, out_dir)

['Salsa20_128_WA_none', 'AES_128_O_default', 'Salsa20_128_O_default', 'Salsa20_256_WA_none', 'AES_128_O_none', 'AES_256_WA_default', 'AES_256_O_none', 'AES_256_WA_none', 'Salsa20_256_O_default', 'AES_128_WA_default', 'Salsa20_128_WA_default', 'AES_128_WA_none', 'Salsa20_256_O_none', 'AES_256_O_default', 'Salsa20_128_O_none', 'Salsa20_256_WA_default']
['mem-loads,mem-stores,cache-references,LLC-load-misses', 'uops_executed_port.port_0,uops_executed_port.port_1,uops_executed_port.port_5,uops_executed_port.port_6', 'instructions,br_inst_retired.all_branches,avx_insts.all,block:block_rq_issue', 'uops_executed_port.port_2,uops_executed_port.port_3,uops_executed_port.port_4,uops_executed_port.port_7']


In [165]:
def get_pname2(idx):
    if idx == 1:
        return "fscan"
    if idx == 2:
        return "compress"
    if idx == 3:
        return "transfer"

def get_count2(phasename, path):
    count = 0
    for file in os.listdir(path):
        if phasename in file:
            count+=1
    return (count)

def get_fname2(phase, path, original):
    oparts = original.split('_')
    if phase == 'fscan':
        count = get_count2(phase, path)
        return f"{phase}_{count}"
    elif phase == 'compress':
        count = get_count2(f"{phase}_{oparts[0]}_{oparts[1][1:-1]}t", path)
        return f"{phase}_{oparts[0]}_{oparts[1][1:-1]}t_{count}"
    elif phase == 'transfer':
        count = get_count2(f"{phase}_{oparts[2]}_{oparts[1][1:-1]}t", path)
        return f"{phase}_{oparts[2]}_{oparts[1][1:-1]}t_{count}"

def dump_perf_data2(data, ptimes, fname, run_iter, outdir):
    # print(f"{fname} {run_iter}")
    #     for key in data.keys():
    #         print(f"{key}: {len(data[key])}")
    processed_data = {}
    for key in data.keys():
        for pt in ptimes:
            if key in pt:
                phases = ptimes[pt]
        pstart = 0
        for i,time in enumerate(phases):
            if i == 0:
                pstart = time
                continue
            pname = get_pname2(i)
            # pdata = [{t,data[key][t]} for t in data[key] if t<=time and t>=pstart]
            pdata = {}
            for t in data[key]:
                if t<=time and t>=pstart:
                    pdata.update({t:data[key][t]})
            if pname in processed_data:
                processed_data[pname].update({key:pdata})
            else:
                processed_data[pname] = {key:pdata}
            pstart = time
    print(fname)
    # print(processed_data)
    for ptype in processed_data:
        # ct = get_count(ptype, outdir)
        outfile = get_fname2(ptype, outdir, fname)
        tmin = 100
        tmax = 0
        # print(processed_data[ptype])
        header="time,"
        keylist=[]
        for key in processed_data[ptype]:
            # print(key)
            # print(processed_data[ptype][key])
            tmin = list(processed_data[ptype][key].keys())[0] if list(processed_data[ptype][key].keys())[0] < tmin else tmin
            tmax = list(processed_data[ptype][key].keys())[-1] if list(processed_data[ptype][key].keys())[-1] > tmax else tmax
            keylist.append(key)
            header+=f"{key},"
            
        with open(f"{out_dir}/{outfile}","a") as f:
            f.write(header+'\n')
        print(tmin)
        print(tmax)
        # print(processed_data[ptype])
        for t in np.arange(tmin,tmax+0.09,0.1):
            t = round(t,1)
            vstring = f"{t},"
            for key in keylist:
                # print(processed_data[ptype][key])
                if t in processed_data[ptype][key].keys():
                    val = processed_data[ptype][key][t]
                else:
                    val = 0
                vstring+=f"{val},"
            with open(f"{out_dir}/{outfile}","a") as f:
                f.write(vstring+'\n')


In [167]:
## Perf counters processor

def save2(time, value, metric, data):
    if metric in data.keys():
        data[metric].update({time: value})
    else:
        data.update({metric:{time: value}})
    return data

perf_dir = "../outputs/perf_results"
in_dir = f"{perf_dir}/out_exfil"
out_dir = f"{perf_dir}/out_exfil_parsed"

os.makedirs(out_dir, exist_ok=True)

# for file in os.listdir(perf_dir):
#     if 'phase' in file and 'exfil_' in file:
#         with open(f"{perf_dir}/{file}",'r') as fp:
#             lines = fp.readlines()
#             print(lines[-3:])
#             if 'str(elapsed_time)\n' not in lines[-3:]:
#                 print(file)

runs = []
perf_mets = []
for file in os.listdir(in_dir):
    separator = "_"
    file_parts = file.split('_')
    file_run = separator.join(file_parts[1:4])
    if file_run not in runs:
        runs.append(file_run)
    if file[len(file_run)+7:-2] not in perf_mets:
        perf_mets.append(file[len(file_run)+7:-2])
print(runs)
print(perf_mets)

all_data = {}
for run in runs:
    for i in range(1,5):
        data = {}
        for met in perf_mets:
            file = f"exfil_{run}_{met}_{i}"
            phase_file = (file[:-2]+'_4_phase').replace("{","").replace("}","")
            with open(f"{perf_dir}/{phase_file}",'r') as fp:
                lines = fp.readlines()
                if len(lines) > 4:
                    print(f"Something wrong here:{phase_file}")
                else:
                    p1_time = round(float(lines[0][:-2]),1)
                    p2_time = round(float(lines[1][:-2]),1)
                    p3_time = round(float(lines[2][:-2]),1)
                    p4_time = round(float(lines[3][:-2]),1)
                    # print(f"{p1_time}, {p2_time}, {p3_time}, {p4_time}")
            with open(f"{in_dir}/{file}",'r') as f:
                lines = f.readlines()
                for l in lines[2:]:
                    l=l.replace("<not counted>","0")
                    if "#" in l:
                        continue
                    # print(l)
                    # break
                    parts = l.split()
                    # print(round(float(parts[0]),1))
                    time = round(float(parts[0]),1)
                    # print(time)
                    value = int(parts[1].replace(',',''))
                    metric = parts[2]
                    data = save2(time, value, metric, data)
        # print(f"{run} {i}")
        # for key in data.keys():
        #     print(f"{key}: {len(data[key])}")
        # dump_perf_data2(data, phase_times, run, i, out_dir)

['zstd_{1}_aws', 'gzip_{1}_aws', 'zstd_{8}_sftp', 'gzip_{8}_sftp', 'gzip_{8}_aws', 'zstd_{1}_sftp', 'zstd_{8}_aws', 'gzip_{1}_sftp']
['uops_executed_port.port_0,uops_executed_port.port_1,uops_executed_port.port_5,uops_executed_port.port_6', 'instructions,br_inst_retired.all_branches,avx_insts.all,block:block_rq_issue', 'uops_executed_port.port_2,uops_executed_port.port_3,uops_executed_port.port_4,uops_executed_port.port_7', 'mem-loads,mem-stores,cache-references,LLC-load-misses']


In [178]:
p1count = p2count = p3count = 0

def update_pcounts(fname, path):
    global p1count
    global p2count
    global p3count
    p1count = p2count = p3count = 0
    fsplit = fname.split('_')
    for file in os.listdir(path):
        if 'asymm' in file:
            p1count+=1
        if fsplit[1]+'_'+fsplit[2] in file:
            p2count+=1
        if fsplit[3]+'_'+fsplit[2] in file:
            p3count+=1

def getname(fname, phase):
    global p1count
    global p2count
    global p3count
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"asymm_{p1count}"
        case 2:
            return f"symm_{fsplit[1]}_{fsplit[2]}b_{p2count}"
        case 3:
            return f"transfer_{fsplit[3]}_{fsplit[2]}t_{p3count}"
    
in_dir = '../outputs/v4_results/out_exec/'
out_dir = '../outputs/v4_results/out_exec_parsed/'

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    # print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        update_pcounts(file, out_dir)
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if ',54321,' in l:
                if 'Port unreachable' not in l:
                    readout = False
                    continue
                if phase > 0:
                    out_fname = getname(file, phase)
                    # print(out_fname)
                    dump(phrases, out_fname, out_dir)
                    # print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 3:
                    break
            elif readout:
                phrases.append(l)
        if phase == 2:
            out_fname = getname(file, phase)
            # print(out_fname)
            dump(phrases, out_fname, out_dir)
            # print(f"Wrote {len(phrases)} to {out_fname}")

In [179]:
p1count = p2count = p3count = 0

def update_pcounts(fname, path):
    global p1count
    global p2count
    global p3count
    p1count = p2count = p3count = 0
    fsplit = fname.split('_')
    for file in os.listdir(path):
        if 'fscan' in file:
            p1count+=1
        if fsplit[1]+'_'+fsplit[2] in file:
            p2count+=1
        if fsplit[3]+'_'+fsplit[2] in file:
            p3count+=1

def getname(fname, phase):
    global p1count
    global p2count
    global p3count
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"fscan_{p1count}"
        case 2:
            if "none" in fname:
                return 'skipdump'
            return f"compress_{fsplit[1]}_{fsplit[2]}t_{p2count}"
        case 3:
            return f"transfer_{fsplit[3]}_{fsplit[2]}t_{p3count}"
    
in_dir = '../outputs/v4_results/out_exfil/'
out_dir = '../outputs/v4_results/out_exfil_parsed/'

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        update_pcounts(file, out_dir)
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if ',54321,' in l:
                if 'Port unreachable' not in l:
                    readout = False
                    continue
                if phase > 0:
                    out_fname = getname(file, phase)
                    print(out_fname)
                    dump(phrases, out_fname, out_dir)
                    print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 4:
                    break
            elif readout:
                phrases.append(l)
        if phase == 3:
            out_fname = getname(file, phase)
            print(out_fname)
            dump(phrases, out_fname, out_dir)
            print(f"Wrote {len(phrases)} to {out_fname}")

exfil_gzip_8_sftp_5
fscan_0
Wrote 6 to fscan_0
compress_gzip_8t_0
Wrote 8356 to compress_gzip_8t_0
transfer_sftp_8t_0
Wrote 206743 to transfer_sftp_8t_0
exfil_none_8_sftp_5
fscan_1
Wrote 7 to fscan_1
skipdump
Wrote 0 to skipdump
transfer_sftp_8t_1
Wrote 581880 to transfer_sftp_8t_1
exfil_gzip_8_aws_1
fscan_2
Wrote 9 to fscan_2
compress_gzip_8t_1
Wrote 1590 to compress_gzip_8t_1
transfer_aws_8t_0
Wrote 109892 to transfer_aws_8t_0
exfil_none_1_aws_1
fscan_3
Wrote 5 to fscan_3
skipdump
Wrote 0 to skipdump
transfer_aws_1t_0
Wrote 70965 to transfer_aws_1t_0
exfil_zstd_1_sftp_3
fscan_4
Wrote 8 to fscan_4
compress_zstd_1t_0
Wrote 3399 to compress_zstd_1t_0
transfer_sftp_1t_0
Wrote 224555 to transfer_sftp_1t_0
exfil_zstd_1_sftp_4
fscan_5
Wrote 7 to fscan_5
compress_zstd_1t_1
Wrote 1821 to compress_zstd_1t_1
transfer_sftp_1t_1
Wrote 206600 to transfer_sftp_1t_1
exfil_zstd_1_aws_1
fscan_6
Wrote 5 to fscan_6
compress_zstd_1t_2
Wrote 1193 to compress_zstd_1t_2
transfer_aws_1t_1
Wrote 72812 to tran

In [8]:
def get_pname(idx):
    if idx == 1:
        return "index"
    if idx == 2:
        return "browse"

def get_count(phasename, path):
    count = 1
    for file in os.listdir(path):
        if phasename in file:
            count+=1
    return (count)

def get_fname(phase, path, original):
    if phase == "all":
        return original
    oparts = original.split('_')
    if phase == 'index':
        count = get_count(phase, path)
        return f"{phase}_hardware_{count}"
    elif phase == 'browse':
        count = get_count(phase, path)
        return f"{phase}_hardware_{count}"

def dump_perf_data(data, ptimes, outfile, outdir):
    processed_data = {}
    if not ptimes:
        processed_data={"all":data}
    else:
        for key in data.keys():
            # print(key)
            for pt in ptimes:
                # print(ptimes)
                if key in pt:
                    phases = ptimes[pt]
            pstart = 0
            for i,time in enumerate(phases):
                if i == 0:
                    pstart = time
                    continue
                pname = get_pname(i)
                pdata = {}
                for t in data[key]:
                    if t<=time and t>=pstart:
                        pdata.update({t:data[key][t]})
                if pname in processed_data:
                    processed_data[pname].update({key:pdata})
                else:
                    processed_data[pname] = {key:pdata}
                pstart = time
    print(outfile)
    for ptype in processed_data:
        # outfile = f"{fname}_{run_iter}"
        outfile = get_fname(ptype, outdir, outfile)
        tmin = 100
        tmax = 0
        header="time,"
        keylist=[]
        for key in processed_data[ptype]:
            tmin = list(processed_data[ptype][key].keys())[0] if list(processed_data[ptype][key].keys())[0] < tmin else tmin
            tmax = list(processed_data[ptype][key].keys())[-1] if list(processed_data[ptype][key].keys())[-1] > tmax else tmax
            keylist.append(key)
            header+=f"{key},"
            
        with open(f"{out_dir}/{outfile}","a") as f:
            f.write(header+'\n')
        # print(header)
        for t in np.arange(tmin,tmax+0.09,0.1):
            t = round(t,1)
            vstring = f"{t},"
            for key in keylist:
                if t in processed_data[ptype][key].keys():
                    val = processed_data[ptype][key][t]
                else:
                    val = 0
                vstring+=f"{val},"
            with open(f"{out_dir}/{outfile}","a") as f:
                f.write(vstring+'\n')
        # print(vstring+'\n')

In [11]:
## Perf counters processor
def save(time, value, metric, data):
    if metric in data.keys():
        data[metric].update({time: value})
    else:
        data.update({metric:{time: value}})
    return data

prog_name="recon"
prog_dir=f"out_{prog_name}"
run_idx=1
run_count=3
has_phase = False

stats_dir = "/proj/spark-cluster-PG0/ransomware-setups/ransomware"
in_dir = f"{stats_dir}/outputs/perf_results/{prog_dir}"
out_dir = f"{stats_dir}/outputs/perf_results/parsed/{prog_dir}_parsed/"
phase_dir = f"{stats_dir}/outputs/perf_results/{prog_dir}/hw_phase_files/"

os.makedirs(out_dir, exist_ok=True)
runs = []
perf_mets = []
file_prefix = f"{prog_name}"
for file in os.listdir(in_dir):
    separator = "_"
    file_parts = file.split('_')
    if run_idx == -1:
        file_run = "NORUN"
    else:
        file_run = file_parts[run_idx]
    if file_run not in runs:
        runs.append(file_run)
    if separator.join(file_parts[2:-1]) not in perf_mets:
        perf_mets.append(separator.join(file_parts[2:-1]))
print(runs)
print(perf_mets)

if run_idx == -3:
    add_fname = "_r"
else:
    add_fname = ""

all_data = {}
for idx,run in enumerate(runs):
    for i in range(1,run_count+1):
        data = {}
        phase_times = {}
        for met in perf_mets:
            if run == "NORUN":
                file = f"{file_prefix}_{met}_{i}"
            else:
                file = f"{file_prefix}_{run}{add_fname}_{met}_{i}"
            if has_phase:
                phase_file = file+'_phase'
                with open(f"{phase_dir}/{phase_file}",'r') as fp:
                    lines = fp.readlines()
                    if len(lines) > 3:
                        print(f"Something wrong here:{phase_file}")
                    else:
                        p_time = []
                        for l in lines:
                            # fl_num = round(float(l[:-2]),2)
                            # if (fl_num*10)%1 > 0.4:
                            #     fl_num = int(fl_num*10+1)/10
                            timeparts = l.split(':')
                            fl_num = int(timeparts[0])*60+int(timeparts[1])
                            p_time.append(fl_num)
                        phase_times[met] = p_time
            with open(f"{in_dir}/{file}",'r') as f:
                lines = f.readlines()
                for l in lines[2:]:
                    l=l.replace("<not counted>","0")
                    if "#" in l:
                        continue
                    parts = l.split()
                    time = round(float(parts[0]),1)
                    value = int(parts[1].replace(',',''))
                    metric = parts[2]
                    data = save(time, value, metric, data)
        if run == "NORUN":
            outfile = f"{file_prefix}_{i}"
        else:
            outfile = f"{file_prefix}_{run}_{i}"
        dump_perf_data(data, phase_times, outfile, out_dir)

['system', 'net', 'mount']
['mem-loads,mem-stores,cache-references,LLC-load-misses', 'uops_executed_port.port_0,uops_executed_port.port_1,uops_executed_port.port_5,uops_executed_port.port_6', 'instructions,br_inst_retired.all_branches,avx_insts.all,block:block_rq_issue', 'uops_executed_port.port_2,uops_executed_port.port_3,uops_executed_port.port_4,uops_executed_port.port_7']
recon_system_1
recon_system_2
recon_system_3
recon_net_1
recon_net_2
recon_net_3
recon_mount_1
recon_mount_2
recon_mount_3
